In [1]:
# For Google Cloud
from google.cloud import storage
import google.auth

client = storage.Client()
bucket = client.get_bucket('gd-gcp-healthcare-search')
dataset_blob = bucket.blob('qa_ncbi_dataset.tsv')

dataset_file_name = 'data/qa_ncbi_dataset.tsv'
dataset_blob.download_to_filename(dataset_file_name)

In [2]:
import sys
import numpy as np

import math
import scipy
from scipy.sparse import hstack
import pandas as pd
from tqdm import tqdm
import re

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import GroupShuffleSplit, train_test_split
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
import logging

In [3]:
import pecos
from pecos.utils.featurization.text.preprocess import Preprocessor
from pecos.xmc.xtransformer.model import XTransformer, XLinearModel
from pecos.xmc.xtransformer.module import MLProblemWithText
from pecos.utils import smat_util
from pecos.xmc import Indexer, LabelEmbeddingFactory
from pecos.xmc.xtransformer.matcher import TransformerMatcher

from pecos.utils import logging_util
logging_util.setup_logging_config(level=2) # LOGGING CONFIG

In [4]:
from sentence_transformers import models, losses, SentenceTransformer

### Bert vectorizer class

In [5]:
class BertVectorizer:

    def __init__(self, model_name, device=None, batch_size=4):
        # Load pre-trained model tokenizer (vocabulary-multilingual)
        self.model = SentenceTransformer(model_name, 
                                         device=device)
        
        self.device = device
        self.batch_size = batch_size

    @property
    def max_seq_length(self):
        return self.model.max_seq_length

    def fit(self):
        pass

    def transform(self, sentences, return_sparse=False):
        sentences_vectors = self.model.encode(sentences,
                                              batch_size=self.batch_size, 
                                              normalize_embeddings=True,
                                              show_progress_bar=True,
                                              device=self.device)
        
        if return_sparse:
            sentences_vectors = scipy.sparse.csr_matrix(sentences_vectors, 
                                                        dtype=np.float32)

        return sentences_vectors

### Load dataset

In [6]:
dataset_filename = None
try:
    credentials, project = google.auth.default()
    dataset_filename = dataset_file_name # GCP
except:
    if 'google.colab' in sys.modules:
        dataset_filename = '/content/drive/MyDrive/qa_ncbi_dataset.tsv' # google colab
    else:
        dataset_filename = 'data/qa_ncbi_dataset.tsv' # local run
        
df = pd.read_csv(dataset_filename, sep='\t')
df.shape

(29319, 15)

In [7]:
df.question_id.nunique()

4040

### Split into dataframes

In [8]:
# Paper DF # Z
papers_df = df[['snippet_url', 'title', 'abstract']] \
    .drop_duplicates() \
    .reset_index(drop=True).reset_index() \
    .rename(columns={'index': 'paper_id'})
papers_df['title_abstract'] = papers_df['title'] + '. ' + papers_df['abstract']

df = df.merge(papers_df[['paper_id', 'snippet_url']])
papers_df = papers_df.set_index('paper_id')

# # Create paper id from url
# df['paper_id'] = df.snippet_id.str.split('_')
# df['paper_id'] = df['paper_id'].apply(lambda a: a[0]).astype(int)
papers_df.sample()

,snippet_url,title,abstract,title_abstract
paper_id,,,,
25006,http://www.ncbi.nlm.nih.gov/pubmed/22553235,"Detection of NDM-1, VIM-1, KPC, OXA-48, and OX...",Matrix-assisted laser desorption ionization-ti...,"Detection of NDM-1, VIM-1, KPC, OXA-48, and OX..."


In [9]:
# question-paper matrix # Y
question_paper_df = pd.crosstab(df['question_id'], df['paper_id']).astype(int)
question_paper_df = question_paper_df[sorted(question_paper_df.columns)]
question_paper_df.sample()

paper_id,0,1,2,3,4,5,6,7,8,9,...,27794,27795,27796,27797,27798,27799,27800,27801,27802,27803
question_id,,,,,,,,,,,,,,,,,,,,,
5e42d1a748dab47f26000010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Question df # X
questions_df = df[['question_id', 'question_text']] \
    .drop_duplicates('question_id') \
    .set_index('question_id').loc[question_paper_df.index]
questions_df.sample()

,question_text
question_id,
54e0c3e71388e8454a000013,Which enzyme is inhibited by Orteronel?


### Train-test splitting

In [11]:
train_question_ids, valid_question_ids = train_test_split(questions_df.index, 
                                                          test_size=0.1,
                                                          random_state=5)
print(f'Train: {len(train_question_ids)}')
print(f'Valid: {len(valid_question_ids)}')

Train: 3636
Valid: 404


In [12]:
# Questions splitting
train_questions_df = questions_df.loc[train_question_ids]
valid_questions_df = questions_df.loc[valid_question_ids]

# Question-Paper matrix splitting
train_question_paper_df = question_paper_df.loc[train_question_ids]
train_question_paper_csr = scipy.sparse.csr_matrix(train_question_paper_df.values, dtype=np.float32)
valid_question_paper_df = question_paper_df.loc[valid_question_ids]
valid_question_paper_csr = scipy.sparse.csr_matrix(valid_question_paper_df.values, dtype=np.float32)

### Bert vectorization

In [13]:
paper_column = 'title' # title_abstract / title / abstract

# bert_model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
# bert_model_name = 'dmis-lab/biobert-v1.1'
# bert_model_name = 'sentence-transformers/roberta-large-nli-stsb-mean-tokens' # Don't fit into gpu memory
# bert_model_name = 'sentence-transformers/roberta-base-nli-mean-tokens' # Bad quality

bert_model_name = 'sentence-transformers/bert-base-nli-cls-token'
# bert_model_name = 'sentence-transformers/bert-base-nli-mean-tokens'

vectorizer = BertVectorizer(model_name=bert_model_name, 
                            device='cuda',
                            batch_size=512)
print(f'Max sentence size: {vectorizer.max_seq_length}')

02/21/2022 14:51:05 - INFO - sentence_transformers.SentenceTransformer - Load pretrained SentenceTransformer: sentence-transformers/bert-base-nli-cls-token


Max sentence size: 128


In [14]:
train_questions_csr = vectorizer.transform(train_questions_df.question_text.values,
                                               return_sparse=True)
valid_questions_csr = vectorizer.transform(valid_questions_df.question_text.values,
                                               return_sparse=True)
papers_csr = vectorizer.transform(papers_df[paper_column].values,
                                      return_sparse=True)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/55 [00:00<?, ?it/s]

### Hyperparameter tuning with Bayesian Optimization

In [15]:
# Creating cluster chain during training
cluster_chain = None

In [16]:
# Create train and valid ML problems
train_prob = MLProblemWithText(
    X_text=train_questions_df.question_text.values.tolist(), 
    Y=train_question_paper_csr, 
    X_feat=train_questions_csr
)

valid_prob = MLProblemWithText(
    X_text=valid_questions_df.question_text.values.tolist(), 
    Y=valid_question_paper_csr, 
    X_feat=valid_questions_csr
)

In [17]:
from bayes_opt import BayesianOptimization

In [18]:
def black_box_function(learning_rate, 
                       hidden_dropout_prob, 
                       weight_decay, 
                       adam_epsilon):
    
    train_params_dict = {
        'do_fine_tune': True,
        'fix_clustering': False,
        'matcher_params_chain': {
            'model_shortcut': bert_model_name,
            'loss_function': 'squared-hinge',
            'batch_size': 16,
            'max_no_improve_cnt': 50,
            'num_train_epochs': TRAIN_EPOCHS,
            'use_gpu': True,
            'logging_steps': 200,
            'save_steps': 200,
            'bootstrap_method': 'linear',
            'gradient_accumulation_steps': 1,
            'loss_function': 'squared-hinge',

            'learning_rate': learning_rate, 
            'hidden_dropout_prob': hidden_dropout_prob,
            'weight_decay': weight_decay,
            'adam_epsilon': adam_epsilon,
        },
    }
    
    predict_params_dict = {
        'only_topk': 10,
        'truncate_length': 128
    }
    
    train_params = XTransformer.TrainParams.from_dict(train_params_dict, recursive=True)
    pred_params = XTransformer.PredParams.from_dict(predict_params_dict, recursive=True)
    
    xtf = XTransformer.train(
        prob=train_prob,
        clustering=cluster_chain,
        val_prob=valid_prob,
        train_params=train_params,
        pred_params=pred_params,
        y_feat=papers_csr
    )
    
    predicted_question_paper_csr = xtf.predict(X_text=valid_questions_df.question_text.values.tolist(), 
                                               X_feat=valid_questions_csr,
                                               beam_size=10,
                                               only_topk=10)
    
    metric = smat_util.Metrics.generate(valid_question_paper_csr, 
                                    predicted_question_paper_csr, 
                                    topk=10)
    print(metric)
    
    prec = metric.prec[0] # Take first precision
    
    return prec

In [19]:
TRAIN_EPOCHS = 3

pbounds = {
    'learning_rate': (1e-8, 1e-2),
    'hidden_dropout_prob': (0, 0.65),
    'weight_decay': (1e-3, 0.2),
    'adam_epsilon': (1e-8, 1e-3)
}

optimizer = BayesianOptimization(f=black_box_function,
                                 pbounds=pbounds, 
                                 verbose=2,
                                 random_state=5)

In [20]:
optimizer.maximize(init_points = 5, n_iter = 20)
print("Best result: {}; f(x) = {}.".format(optimizer.max["params"], optimizer.max["target"]))

|   iter    |  target   | adam_e... | hidden... | learni... | weight... |
-------------------------------------------------------------------------


02/21/2022 14:51:40 - INFO - pecos.xmc.xtransformer.model - Hierarchical label tree: [2, 32, 512, 27804]
02/21/2022 14:51:40 - INFO - pecos.xmc.xtransformer.model - Fine-tune Transformers with nr_labels=[2, 32, 512, 27804]
02/21/2022 14:51:40 - INFO - pecos.xmc.xtransformer.model - Fine-tuning XR-Transformer with tfn at level 0, nr_labels=2, avr_M_nnz=2
02/21/2022 14:51:44 - INFO - pecos.xmc.xtransformer.matcher - Downloaded sentence-transformers/bert-base-nli-cls-token model from s3.
02/21/2022 14:51:44 - INFO - pecos.xmc.xtransformer.matcher - ***** Encoding data len=3636 truncation=128*****
02/21/2022 14:51:44 - INFO - pecos.xmc.xtransformer.matcher - ***** Finished with time cost=0.3824124336242676 *****
02/21/2022 14:51:44 - INFO - pecos.xmc.xtransformer.matcher - trn tensors saved to /tmp/tmpife9xawj/X_trn.pt
02/21/2022 14:51:44 - INFO - pecos.xmc.xtransformer.matcher - ***** Encoding data len=404 truncation=128*****
02/21/2022 14:51:44 - INFO - pecos.xmc.xtransformer.matcher - *

prec   = 1.73 1.73 1.73 1.61 1.58 1.49 1.31 1.21 1.10 1.01
recall = 0.73 1.00 1.40 1.53 1.71 1.82 1.85 1.88 1.90 1.91
|  1        |  0.01733  |  0.000222 |  0.566    |  0.002067 |  0.1838   |


02/21/2022 15:00:20 - INFO - pecos.xmc.xtransformer.model - Hierarchical label tree: [2, 32, 512, 27804]
02/21/2022 15:00:20 - INFO - pecos.xmc.xtransformer.model - Fine-tune Transformers with nr_labels=[2, 32, 512, 27804]
02/21/2022 15:00:20 - INFO - pecos.xmc.xtransformer.model - Fine-tuning XR-Transformer with tfn at level 0, nr_labels=2, avr_M_nnz=2
02/21/2022 15:00:24 - INFO - pecos.xmc.xtransformer.matcher - Downloaded sentence-transformers/bert-base-nli-cls-token model from s3.
02/21/2022 15:00:24 - INFO - pecos.xmc.xtransformer.matcher - ***** Encoding data len=3636 truncation=128*****
02/21/2022 15:00:24 - INFO - pecos.xmc.xtransformer.matcher - ***** Finished with time cost=0.4100492000579834 *****
02/21/2022 15:00:24 - INFO - pecos.xmc.xtransformer.matcher - trn tensors saved to /tmp/tmpiqdm2k78/X_trn.pt
02/21/2022 15:00:24 - INFO - pecos.xmc.xtransformer.matcher - ***** Encoding data len=404 truncation=128*****
02/21/2022 15:00:24 - INFO - pecos.xmc.xtransformer.matcher - *

prec   = 1.73 1.73 1.73 1.61 1.58 1.49 1.31 1.21 1.10 1.01
recall = 0.73 1.00 1.40 1.53 1.71 1.82 1.85 1.88 1.90 1.91
|  2        |  0.01733  |  0.000488 |  0.3976   |  0.007659 |  0.1042   |


...

prec   = 1.73 1.73 1.73 1.61 1.58 1.49 1.31 1.21 1.10 1.01
recall = 0.73 1.00 1.40 1.53 1.71 1.82 1.85 1.88 1.90 1.91
|  23       |  0.01733  |  0.000155 |  0.1906   |  0.004854 |  0.1612   |


02/21/2022 18:13:06 - INFO - pecos.xmc.xtransformer.model - Hierarchical label tree: [2, 32, 512, 27804]
02/21/2022 18:13:06 - INFO - pecos.xmc.xtransformer.model - Fine-tune Transformers with nr_labels=[2, 32, 512, 27804]
02/21/2022 18:13:06 - INFO - pecos.xmc.xtransformer.model - Fine-tuning XR-Transformer with tfn at level 0, nr_labels=2, avr_M_nnz=2
02/21/2022 18:13:09 - INFO - pecos.xmc.xtransformer.matcher - Downloaded sentence-transformers/bert-base-nli-cls-token model from s3.
02/21/2022 18:13:09 - INFO - pecos.xmc.xtransformer.matcher - ***** Encoding data len=3636 truncation=128*****
02/21/2022 18:13:09 - INFO - pecos.xmc.xtransformer.matcher - ***** Finished with time cost=0.21700453758239746 *****
02/21/2022 18:13:10 - INFO - pecos.xmc.xtransformer.matcher - trn tensors saved to /tmp/tmpapupxink/X_trn.pt
02/21/2022 18:13:10 - INFO - pecos.xmc.xtransformer.matcher - ***** Encoding data len=404 truncation=128*****
02/21/2022 18:13:10 - INFO - pecos.xmc.xtransformer.matcher - 

prec   = 1.73 1.61 1.82 1.79 1.68 1.53 1.34 1.18 1.05 0.94
recall = 0.39 0.53 1.10 1.39 1.52 1.60 1.62 1.62 1.62 1.62
|  24       |  0.01733  |  0.000639 |  0.5404   |  0.001087 |  0.1207   |


02/21/2022 18:21:58 - INFO - pecos.xmc.xtransformer.model - Hierarchical label tree: [2, 32, 512, 27804]
02/21/2022 18:21:58 - INFO - pecos.xmc.xtransformer.model - Fine-tune Transformers with nr_labels=[2, 32, 512, 27804]
02/21/2022 18:21:58 - INFO - pecos.xmc.xtransformer.model - Fine-tuning XR-Transformer with tfn at level 0, nr_labels=2, avr_M_nnz=2
02/21/2022 18:22:02 - INFO - pecos.xmc.xtransformer.matcher - Downloaded sentence-transformers/bert-base-nli-cls-token model from s3.
02/21/2022 18:22:02 - INFO - pecos.xmc.xtransformer.matcher - ***** Encoding data len=3636 truncation=128*****
02/21/2022 18:22:02 - INFO - pecos.xmc.xtransformer.matcher - ***** Finished with time cost=0.3708474636077881 *****
02/21/2022 18:22:02 - INFO - pecos.xmc.xtransformer.matcher - trn tensors saved to /tmp/tmp0d5x386w/X_trn.pt
02/21/2022 18:22:02 - INFO - pecos.xmc.xtransformer.matcher - ***** Encoding data len=404 truncation=128*****
02/21/2022 18:22:02 - INFO - pecos.xmc.xtransformer.matcher - *

prec   = 1.98 1.86 1.73 1.61 1.58 1.49 1.31 1.21 1.10 1.01
recall = 0.74 1.02 1.40 1.53 1.71 1.82 1.85 1.88 1.90 1.91
|  25       |  0.0198   |  0.000551 |  0.1092   |  0.003251 |  0.1751   |
Best result: {'adam_epsilon': 0.0006027209623799, 'hidden_dropout_prob': 0.07526068743757297, 'learning_rate': 0.002661347396322538, 'weight_decay': 0.06856982448680521}; f(x) = 0.022277227722772276.


### Train

In [21]:
# Creating cluster chain during training
cluster_chain = None

In [22]:
# Create train and valid ML problems
train_prob = MLProblemWithText(
    X_text=train_questions_df.question_text.values.tolist(), 
    Y=train_question_paper_csr, 
    X_feat=train_questions_csr
)

valid_prob = MLProblemWithText(
    X_text=valid_questions_df.question_text.values.tolist(), 
    Y=valid_question_paper_csr, 
    X_feat=valid_questions_csr
)

In [23]:
train_params_dict = {
    'do_fine_tune': True,
    'fix_clustering': False,
    'matcher_params_chain': {
        'model_shortcut': bert_model_name,
        'loss_function': 'squared-hinge',
        'batch_size': 16,
        'max_no_improve_cnt': 50,
        'num_train_epochs': 3,
        'use_gpu': True,
        'logging_steps': 200,
        'save_steps': 200,
        'learning_rate': 1e-3,
        'bootstrap_method': 'linear',
        'hidden_dropout_prob': 0.45
    },
}

predict_params_dict = {
    'only_topk': 10,
    'truncate_length': 300
}

train_params = XTransformer.TrainParams.from_dict(train_params_dict, recursive=True)
pred_params = XTransformer.PredParams.from_dict(predict_params_dict, recursive=True)

In [24]:
xtf = XTransformer.train(
    prob=train_prob,
    clustering=cluster_chain,
    val_prob=valid_prob,
    train_params=train_params,
    pred_params=pred_params,
    y_feat=papers_csr
)

02/21/2022 18:30:44 - INFO - pecos.xmc.xtransformer.model - Hierarchical label tree: [2, 32, 512, 27804]
02/21/2022 18:30:44 - INFO - pecos.xmc.xtransformer.model - Fine-tune Transformers with nr_labels=[2, 32, 512, 27804]
02/21/2022 18:30:44 - INFO - pecos.xmc.xtransformer.model - Fine-tuning XR-Transformer with tfn at level 0, nr_labels=2, avr_M_nnz=2
02/21/2022 18:30:48 - INFO - pecos.xmc.xtransformer.matcher - Downloaded sentence-transformers/bert-base-nli-cls-token model from s3.
02/21/2022 18:30:48 - INFO - pecos.xmc.xtransformer.matcher - ***** Encoding data len=3636 truncation=300*****
02/21/2022 18:30:48 - INFO - pecos.xmc.xtransformer.matcher - ***** Finished with time cost=0.5879473686218262 *****
02/21/2022 18:30:48 - INFO - pecos.xmc.xtransformer.matcher - trn tensors saved to /tmp/tmplbg79d6d/X_trn.pt
02/21/2022 18:30:48 - INFO - pecos.xmc.xtransformer.matcher - ***** Encoding data len=404 truncation=300*****
02/21/2022 18:30:49 - INFO - pecos.xmc.xtransformer.matcher - *

In [25]:
xtf.save("models/xmc_transformer_4e")

02/21/2022 18:47:24 - INFO - pecos.xmc.xtransformer.model - Parameters saved to models/xmc_transformer_4e/param.json
02/21/2022 18:47:25 - INFO - pecos.xmc.xtransformer.model - Model saved to models/xmc_transformer_4e


### Validation

In [26]:
predicted_question_paper_csr = xtf.predict(X_text=valid_questions_df.question_text.values.tolist(), 
                                           X_feat=valid_questions_csr,
                                           beam_size=10,
                                           only_topk=10)

02/21/2022 18:47:25 - INFO - pecos.utils.torch_util - Setting device to cuda, number of active GPUs: 1
02/21/2022 18:47:25 - INFO - pecos.xmc.xtransformer.matcher - ***** Encoding data len=404 truncation=300*****
02/21/2022 18:47:25 - INFO - pecos.xmc.xtransformer.matcher - ***** Finished with time cost=0.05874300003051758 *****
02/21/2022 18:47:25 - INFO - pecos.xmc.xtransformer.matcher - Predict on input text tensors(torch.Size([404, 300]))


In [27]:
metric = smat_util.Metrics.generate(valid_question_paper_csr, 
                                    predicted_question_paper_csr, 
                                    topk=10)
print(metric)

prec   = 1.73 1.73 1.73 1.61 1.58 1.49 1.31 1.21 1.10 1.01
recall = 0.73 1.00 1.40 1.53 1.71 1.82 1.85 1.88 1.90 1.91


### Checking

In [28]:
def get_papers_by_text(text, 
                       vectorizer,
                       number_of_papers=10, 
                       check_correctness=False):
  
    vector_text_csr = vectorizer.transform([text])

    papers_csr = xlm.predict(vector_text_csr,
                             beam_size=4,
                             only_topk=number_of_papers)

    prediction = papers_csr.toarray()
    paper_labels = np.nonzero(prediction)[1].tolist()

    # print()
    # print('Found papers: ', len(paper_labels))
    # print(paper_labels)

    paper_logists = np.take(prediction, paper_labels).tolist()

    result = papers_df.loc[paper_labels, ['title', 'abstract', 'snippet_url']]
    result['score'] = paper_logists
    result = result.sort_values('score', ascending=False)

    if check_correctness:
        correct_papers = df.loc[df.question_text == text, 'paper_id'].unique()
        print(f'\nTrue number of papers: {len(correct_papers)}')
        result['correctness'] = result.index.isin(correct_papers)

    return result

In [29]:
correct_examples = [
    'Which genome browser database for DNA shape annotations is available?',
    'Has strimvelis been approved by the European Medicines Agency?',
    'What is the target of Volanesorsen?',
    'Is eptinezumab a small molecule?'
]

interesting_examples = [
    'For which type of diabetes can empagliflozin be used?',
    "Is Semagacestat effective for Alzheimer's Disease?",
    'Which gene is associated with Muenke syndrome?'     
]

In [30]:
get_papers_by_text('Which gene is associated with Muenke syndrome?',
                   vectorizer=vectorizer, 
                   number_of_papers=30,
                   check_correctness=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

NameError: name 'xlm' is not defined

In [ ]:
# Random example
temp = valid_questions_df.question_text.sample().values[0]
print(temp)

get_papers_by_text(temp,
                   vectorizer=vectorizer,
                   number_of_papers=30,
                   check_correctness=True)

### Highlight answers

In [ ]:
question_text = 'Which genome browser database for DNA shape annotations is available?'

In [ ]:
result = get_papers_by_text(
    question_text,
    vectorizer=vectorizer, 
    number_of_papers=30,
    check_correctness=True
)

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("ozcangundes/T5-base-for-BioQA")
model = T5ForConditionalGeneration.from_pretrained("ozcangundes/T5-base-for-BioQA")

def get_answer(question, context):
    source_encoding=tokenizer(
        question,
        context,
        max_length=512,
        padding="max_length",
        truncation="only_second",
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt")

    generated_ids=model.generate(
        input_ids=source_encoding["input_ids"],
        attention_mask=source_encoding["attention_mask"])

    preds=[tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for gen_id in generated_ids]
    
    return "".join(preds)

# To highlight the answers in the results
def highlight_selected_text(row):
    abstract = row["abstract"]
    title = row["title"]
    ext = []
    ext.append(row["abstract_answer"])
    ext.append(row['title_answer'])
    # for k, v in color.items():
    for k in ext:
      if len(k) > 2:
        abstract = abstract.replace(k, f'<span style="color: red; background-color: yellow; font-weight: bold">{k}</span>')
        title = title.replace(k, f'<span style="color: red; background-color: yellow; font-weight: bold">{k}</span>')
    return abstract, title

In [ ]:
result['abstract_answer'] = result['abstract'].apply(lambda x: get_answer(question_text, x))
result['title_answer'] = result['title'].apply(lambda x: get_answer(question_text, x))

In [ ]:
from IPython.core.display import HTML
result['abstract'], result['title'] = zip(*result.apply(highlight_selected_text, axis=1))
display(HTML(result.sample(10).to_html(escape=False)))